<a href="https://colab.research.google.com/github/TerriaLynnn/Bilibili-Evolved/blob/master/11111.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

%matplotlib inline

In [12]:
def get_rmse_log(model, feature, label, use_gpu):
    if use_gpu:
        feature = feature.cuda()
        label = label.cuda()
    model.eval()
    mse_loss = nn.MSELoss()
    with torch.no_grad():
        pred = model(feature)
    # clipped_pred = pred.clamp(1, float('inf'))
    rmse = (mse_loss(pred, label)).sqrt()
    return rmse.item()

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
delimiter=","
train = pd.read_csv('6w.CSV',delimiter=",",encoding="gbk")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


可以具体看看前面 5 个训练集长什么样子，可以看到，前面都是这个房屋的属性，最后是房屋的价格

In [15]:
train.head()

,课程编号,账号,课程名称,任务点完成率,视频任务点完成率,学生观看视频总时长(分钟),章节测验完成率,章节测验平均分,作业完成率,作业平均分,章节学习次数,签到完成率,讨论总数,综合成绩,任务点完成率等级,视频任务点完成率等级,章节测验完成率等级,作业完成率等级,成绩等级
0,3079,2.02E+11,2020春-《认识武汉》通识教育选修课,100.0,100.0,780.8,0.0,0.0,0.00,0.0,38,0.0,0,97.30,A,A,F,F,A
1,3079,1.62E+11,2020春-《认识武汉》通识教育选修课,100.0,100.0,813.2,0.0,0.0,0.00,0.0,110,0.0,1,96.80,A,A,F,F,A
2,3079,1.72E+11,2020春-《认识武汉》通识教育选修课,100.0,100.0,810.7,0.0,0.0,16.67,571.6,19,0.0,0,96.58,A,A,F,F,A
3,3079,1.72E+11,2020春-《认识武汉》通识教育选修课,100.0,100.0,908.3,0.0,0.0,0.00,0.0,278,0.0,2,96.55,A,A,F,F,A
4,3079,2.02E+11,2020春-《认识武汉》通识教育选修课,100.0,100.0,788.4,0.0,0.0,0.00,0.0,72,0.0,5,96.55,A,A,F,F,A


接着我们可以看看训练集和测试集分别有多少个样本

In [16]:
print('一共有 {} 个训练集样本'.format(train.shape[0]))

一共有 65946 个训练集样本


接着我们开始对数据进行处理，首先我们取出**第二个特征**到**倒数第二个特征**，这些特征作为我们神经网络的输入特征

In [17]:
all_features =train.loc[:, '任务点完成率':'课程积分']
all_features = all_features[:10000]
print(all_features.head())

KeyError: ignored

接着我们需要进行数据标准化，对于所有的数值特征，我们都会减去均值，除以方差

In [ ]:
numeric_feats = all_features.dtypes[all_features.dtypes != "object"].index # 取出所有的数值特征

# 减去均值，除以方差
all_features[numeric_feats] = all_features[numeric_feats].apply(lambda x: (x - x.mean()) 
                                                                / (x.std()))

如果你仔细看看上面的特征，你会发现，除了数值特征之外，还有很多非数值特征，这些特征我们没有办法将其转换成数值表示，所以我们通过 pandas 的内置函数将其转换成种类表示

比如 **MSZoning** 有两种可能，一种是 RL，一种是 RM，那么我们就将这个特征变成两个新的特征，RL 和 RM，如果这个数据在 **MSZoning** 上是 RL，那么 RL 取 1，RM 取 0；反之如果这个特征是 RM，那么 RL 取 0，RM 取 1.

| RL | RM |
|-|-|
| 0 | 1 |
| 1 | 0 |

In [ ]:
all_features = pd.get_dummies(all_features, dummy_na=True)

除此之外，我们会发现整个数据中有一些丢失数据，这些丢失数据都是 'NA'，我们没有办法将这些数据输入到网络中，所以需要对这些丢失数据进行赋值，这里我们将数据的均值填入到丢失数据中

In [ ]:
all_features = all_features.fillna(all_features.mean())
print(all_features.head)

前面我们已经做好了数据的预处理，下面我们将所有的训练集和验证集都取出成为一个 numpy 的数组

In [ ]:
num_train = 8000
print(num_train)
train_features = all_features[:num_train].values.astype(np.float32)
test_features = all_features[num_train:].values.astype(np.float32)

train_labels = train[:num_train]["综合成绩"].values[:, None].astype(np.float32)
test_labels = train[num_train:10000]["综合成绩"].values[:, None].astype(np.float32)
print(train_features.shape)
print(train_labels)
print(test_features.shape)
print(test_labels)

下面需要你来填写代码完成本次的项目

**注意：你只需要完成下面 todo 的部分**

In [ ]:
from torch import nn

def get_model():
    # todo: 使用 nn.Sequential 来构造多层神经网络，注意第一层的输入
    model = nn.Sequential(
        nn.Linear(26, 300),
        nn.ReLU(),
        nn.Linear(300,200),
        nn.ReLU(),
        nn.Linear(200, 100),
        nn.ReLU(),
        nn.Linear(100,1))
    return model

In [ ]:
# 可以调整的超参

batch_size = 64
epochs = 50
use_gpu = False
lr = 1e-3
weight_decay = 1e-5

In [ ]:
criterion = nn.MSELoss()# todo: 使用 mse 作为 loss 函数

在评估模型的时候，为了保证大的价格和小的价格对模型都有着近似相同的影响，我们不会直接使用前面定义的均方误差作为最后的评价函数，我们会对预测的价格和真实的价格取 log，然后计算他们之间均方误差的平方根来作为评价指标，这里的指标我们已经在 `utils.py` 中实现了，感兴趣的同学可以去看看。

In [ ]:
import torch
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader

#from utils import get_rmse_log

In [ ]:
# todo: 将所有的 feature 和 label 都转成 torch 的 Tensor
train_features = torch.from_numpy(train_features)
test_features = torch.from_numpy(test_features)
train_labels = torch.from_numpy(train_labels)
test_labels = torch.from_numpy(test_labels)

In [ ]:
# 构建一个数据的迭代器
def get_data(x, y, batch_size, shuffle):
    dataset = TensorDataset(x, y)
    return DataLoader(dataset, batch_size, shuffle=shuffle, num_workers=4)

In [ ]:
def train_model(model, x_train, y_train, x_valid, y_valid, epochs, lr, weight_decay):
    metric_log = dict()
    metric_log['train_loss'] = list()
    if x_valid is not None:
        metric_log['valid_loss'] = list()
    
    train_data = get_data(x_train, y_train, batch_size, True)
    if x_valid is not None:
        valid_data = get_data(x_valid, y_valid, batch_size, False)
    else:
        valid_data = None
    
    optimizer = torch.optim.Adam(model.parameters(), lr = lr, weight_decay = weight_decay)# todo: 构建优化器，推荐使用 Adam，也可以尝试一下别的优化器
    
    for e in range(epochs):
        # 训练模型
        running_loss = 0
        model.train()
        for data in train_data:
            x, y = data
            if use_gpu:
                x = x.cuda()
                y = y.cuda()
            x = Variable(x)
            y = Variable(y)
            
            # todo: 前向传播
            out = model(x)
            # todo: 计算 loss
            loss = criterion(out, y)
            # todo: 反向传播，更新参数
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        metric_log['train_loss'].append(get_rmse_log(model, x_train, y_train, use_gpu))
        
        # 测试模型
        if x_valid is not None:
            metric_log['valid_loss'].append(get_rmse_log(model, x_valid, y_valid, use_gpu))
            print_str = 'epoch: {}, train loss: {:.3f}, valid loss: {:.3f}'\
            .format(e+1, metric_log['train_loss'][-1], metric_log['valid_loss'][-1])
        else:
            print_str = 'epoch: {}, train loss: {:.3f}'.format(e+1, metric_log['train_loss'][-1])
        if (e + 1) % 10 == 0:
            print(print_str)
            print()

    # =======不要修改这里的内容========
    # 可视化
    figsize = (10, 5)
    fig = plt.figure(figsize=figsize)
    plt.plot(metric_log['train_loss'], color='red', label='train')
    if valid_data is not None:
        plt.plot(metric_log['valid_loss'], color='blue', label='valid')
    plt.legend(loc='best')
    plt.xlabel('epochs')
    plt.ylabel('loss')
    plt.show()

当我们构建好了训练的过程，下面就开始了不断地调参尝试，最后得到一个效果最好的模型

In [ ]:
model = get_model()
train_model(model, train_features, train_labels, test_features, test_labels, epochs, lr, weight_decay)